In [25]:
import math
import requests

In [26]:
def lat_lon_to_tile_epsg4326(lat, lon, zoom):
    # Constrain latitude and longitude
    constrained_lat = max(-89.999999, lat)  # Constrain lat within (-90, 90]
    constrained_lon = 179.999999 if lon == 180 else lon  # Constrain lon to be < 180

    # Define bounds in EPSG:4326
    min_lat = -90
    max_lat = 90
    min_lon = -180
    max_lon = 180

    # Number of tiles at this zoom level
    tile_count_x = math.pow(2, zoom + 1)
    tile_count_y = math.pow(2, zoom)

    # Calculate tile width and height in terms of lat/lon degrees
    tile_width = (max_lon - min_lon) / tile_count_x  # 360 degrees / 2^(zoom+1)
    tile_height = (max_lat - min_lat) / tile_count_y  # 180 degrees / 2^zoom

    # Calculate tile index
    tile_x = math.floor((constrained_lon - min_lon) / tile_width)
    tile_y = math.floor((max_lat - constrained_lat) / tile_height)

    # Pixel position within the tile (256x256 pixels)
    pixel_x = math.floor((((constrained_lon - min_lon) % tile_width) / tile_width) * 256)
    pixel_y = math.floor((((max_lat - constrained_lat) % tile_height) / tile_height) * 256)

    return tile_x, tile_y, pixel_x, pixel_y

In [30]:
def wmt_getfeatureinfo(product, dataset,tag, variable, tile_x, tile_y, i, j, zoom, time, depth=0):
    wmts_url = f"https://wmts.marine.copernicus.eu/teroWmts/?service=WMTS&request=GetFeatureInfo&layer={product}/{dataset}_{tag}/{variable}&tilematrixset=EPSG:4326&tilematrix={zoom}&tilerow={tile_y}&tilecol={tile_x}&i={i}&j={j}&INFOFORMAT=application/json&time={time}&elevation={depth}"

    try:
        response = requests.get(wmts_url)
        response.raise_for_status()
        if response.status_code == 200:
        # Parse the JSON response
            data = response.json()
            print(wmts_url)
            print("JSON Response:", data)
            return data['features'][0]['properties']
        else:
            print("Request failed with status code:", response.status_code)
            print(wmts_url)
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"Other error occurred: {err}")
    except ValueError:
        print("Error parsing JSON response")


In [ ]:
## Example
product= "GLOBAL_ANALYSISFORECAST_PHY_001_024"
dataset = "cmems_mod_glo_phy-thetao_anfc_0.083deg_P1M-m"
tag = "202406"
variable = "thetao"

lon=64.14
lat=-23.03
depth=0
time="2023-05-16"

# Use the highest zoom level possible for best precision
zoom = 10
tile_x, tile_y, pixel_x, pixel_y = lat_lon_to_tile_epsg4326(lat, lon, zoom)

data = wmt_getfeatureinfo(product, dataset, tag, variable, tile_x, tile_y, pixel_x, pixel_y, zoom, time, depth)
data